In [1]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from typing import Optional
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
import dotenv
import os
dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:

gpt_model = "gpt-3.5-turbo-0125"
groq_model = "llama3-8b-8192"
OPENAI_API_KEY = os.getenv("groq_api_key") 
llm = ChatGroq(model=groq_model)

In [32]:
'''
REVIEWS_CSV_PATH = "E:\Data Science\Capstone\data"
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = CSVLoader(file_path=REVIEWS_CSV_PATH)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are a professional stock market analyst and investor "
    "Use the following pieces of retrieved context to answer a question"
    " If you don't know the answer, say that you "
    " stocks which have results better than previous results is good "
    "don't know. Use three sentences maximum and keep the "
    "answer concise.consider only Indian stockmarket "
    "Answer only from the content retreived do not answer anything from outside "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

output_parser = StrOutputParser()
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
'''

'\nREVIEWS_CSV_PATH = "E:\\Data Science\\Capstone\\data"\n# 1. Load, chunk and index the contents of the blog to create a retriever.\nloader = CSVLoader(file_path=REVIEWS_CSV_PATH)\ndocs = loader.load()\n\ntext_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)\nsplits = text_splitter.split_documents(docs)\nvectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())\nretriever = vectorstore.as_retriever()\n\n\n# 2. Incorporate the retriever into a question-answering chain.\nsystem_prompt = (\n    "You are a professional stock market analyst and investor "\n    "Use the following pieces of retrieved context to answer a question"\n    " If you don\'t know the answer, say that you "\n    " stocks which have results better than previous results is good "\n    "don\'t know. Use three sentences maximum and keep the "\n    "answer concise.consider only Indian stockmarket "\n    "Answer only from the content retreived do not answer anything fr

In [4]:
import os
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import SimpleOutputParser
from langchain.llms import OpenAI  # Assuming you're using OpenAI LLM

# Directory containing CSV files
directory_path = "E:\\Data Science\\Capstone\\data"

# Function to load all CSV files from a directory
def load_csv_files_from_directory(directory_path):
    all_docs = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            loader = CSVLoader(file_path=file_path)
            try:
                docs = loader.load()
                all_docs.extend(docs)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
    return all_docs

# Load documents from all CSV files
docs = load_csv_files_from_directory(directory_path)

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Load BERT embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/bert-base-nli-mean-tokens")

# Create a vector store from the chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

# Define the system prompt and create the QA chain
system_prompt = (
    "You are a professional stock market analyst and investor. "
    "Use the following pieces of retrieved context to answer a question. "
    "If you don't know the answer, say that you don't know. "
    "Stocks which have results better than previous results are good. "
    "Consider only the Indian stock market. "
    "Answer only from the content retrieved; do not answer anything from outside. "
    "\n\n{context}"
)

# Create the LLM (assuming OpenAI here, but replace with your LLM)
llm = OpenAI(model="gpt-3.5-turbo")

# Create the chain for answering
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
output_parser = SimpleOutputParser()
stuff_chain = StuffDocumentsChain(prompt=prompt, output_parser=output_parser, llm=llm)


ImportError: cannot import name 'SimpleOutputParser' from 'langchain.output_parsers' (c:\Users\PENOSH YADAV\anaconda3\envs\MGI\Lib\site-packages\langchain\output_parsers\__init__.py)

In [ ]:

# Function to perform retrieval and generate the answer
def answer_question(query):
    # Retrieve relevant documents
    retrieved_docs = retriever.get_relevant_documents(query)
    
    # Combine the documents and run the QA chain
    answer = stuff_chain.run(input_documents=retrieved_docs, input=query)
    return answer

# Example usage
question = "What are the best-performing stocks in the Indian market?"
response = answer_question(question)
print(response)


In [18]:
response = rag_chain.invoke({"input": "give me stocks which could give massive returns"})
output_parser.parse(response["answer"])

'Based on the text, it is mentioned that the Indian market has shown robust growth with a 46% increase in the past year, and earnings are expected to grow by 16% annually. This suggests that many stocks may have reported better financial results than the previous year.\n\nIn particular, the text highlights the Financials and Information Technology sectors, which have gained 1.4% and 3.2%, respectively, over the last 7 days. This implies that some stocks within these sectors may be showing improved financial performance compared to the previous year.\n\nHowever, it is not possible to identify specific stocks that have reported better financial results without further information.'

In [19]:


contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [20]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [21]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "give me stocks which could give massive returns"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "name one stock which meets the criteria?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

I don't know. The provided context does not mention a specific stock that meets the criteria of having better financial results than the previous year.


In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [23]:
conversational_rag_chain.invoke(
    {"input": "stocks in energy sector?"},
    config={
        "configurable": {"session_id": "CAPP"}
    },  
)["answer"]

"I don't know. The text does not mention stocks from the energy sector."

In [24]:
conversational_rag_chain.invoke(
    {"input": "any other stock in energy sector?"},
    config={"configurable": {"session_id": "CAPP"}},
)["answer"]

'I see that NSEI:ONGC is mentioned in the text, which is an energy company.'

In [25]:
for message in store["CAPP"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: stocks in energy sector?

AI: I don't know. The text does not mention stocks from the energy sector.

User: any other stock in energy sector?

AI: I see that NSEI:ONGC is mentioned in the text, which is an energy company.

